In [62]:
import pandas as pd
import s3fs
import numpy as np
import networkx as nx

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [9]:
fs = s3fs.S3FileSystem()

path = fs.glob('s3://zigdata.org/reddit-headlines/*')[-1]
titles = pd.read_parquet(f"s3://{path}")
titles['created'] = pd.to_datetime(titles.created_epoch, unit='s')
titles['extracted'] = pd.to_datetime(titles.extracted_epoch, unit='s')

In [56]:
titles['words'] = titles.title.str.split(' ').apply(len)

In [60]:
titles.groupby('subreddit').words.mean().sort_values(ascending=True).head()

subreddit
me_irl        1.011494
nsfw          3.071197
creepy        4.086957
Unexpected    4.485685
NSFW_GIF      4.505820
Name: words, dtype: float64

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=100)
X = vectorizer.fit_transform(titles.title)
Xc = (X.T * X)
Xc.setdiag(0)
matrix_dense = Xc.todense()
G = nx.from_numpy_matrix(matrix_dense)